## Import des librairies

In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import from_json
from pyspark.sql import Row
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *
from pyspark.ml.feature import HashingTF,Tokenizer,StopWordsRemover
from pyspark.mllib.linalg import Vectors, SparseVector
from pyspark.mllib.util import MLUtils
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pymongo import MongoClient
import pprint as p
import pandas, json, sys
from whatthelang import WhatTheLang
wtl = WhatTheLang()

# Création d'un spark context, d'un sql context et d'un streaming context
sc = SparkContext("local[2]","Commentaires AirBnB")
sc.setLogLevel("FATAL")
sqlContext = SQLContext(sc)
ssc = StreamingContext(sc, 1)
socket_stream = ssc.socketTextStream("localhost",9999)

stopwords = sc.textFile("French_stop_words.txt").collect()


## Réception des données, transformation, prédiction et enregistrement sur MongoDB


In [2]:

# Récupération sur une fenêtre d'1s
flux = socket_stream.window( 1 )

## Chargement du modèle calculé
GBTModel = GradientBoostedTreesModel.load(sc, "modele/myGradientBoostingClassificationModel_SW")

# Traitement sur chaque ligne du commentaire

def TraitementCommentaire(x):
 # Récupération des RDD dans une liste
    x = x.collect()

 # Gestion de la fin du fichier
    if (x == 'Fini'):
        # arrêt des context
        global ssc
        global sqlContext
        global sc
        ssc.stop()
        sqlContext.stop()
        sc.stop()
        print("Le traitement du flux est terminé")
        return 1
            
    # Connection à la base MongoDB (qui tourne sur Docker)
    client = MongoClient('localhost:27017')
    db=client.test.Comments 
    

    # Gestion des différents item de la liste
    for item in x:
        comment_texte = item
        #print(comment_texte)
        
        #transformation en dataframe pandas
        comment = pandas.read_json(comment_texte,typ='series').to_frame().transpose()
        comment['comments']=comment['comments'].replace("\r\n", "")
        comment['comments']=comment['comments'].replace("\n", "")
        comment['comments']=comment['comments'].astype(str)
        # Détection de la langue et ajout au dataframe
        comment['langage']=comment['comments'].apply(wtl.pred_prob)
        comment['langue']=comment['langage'].str[0].str[0].str[0]
        comment['lg_proba']=comment['langage'].str[0].str[0].str[1]
        comment=comment.drop(['langage'],axis=1)
        comment = comment[(comment['lg_proba']>0.8) & (comment['langue']=='fr')]
        print(comment.shape[0])
        test_langue = comment.shape[0]
        if (test_langue > 0) :
            print("test OK")
            #transformation du dataframe Pandas en RDD
            CommentDF = sqlContext.createDataFrame(comment[['id','comments']])
            CommentDF.registerTempTable("CommentDF")
            
            tokenizer = Tokenizer(inputCol="comments", outputCol="words")
            wordsData = tokenizer.transform(CommentDF)
            remover = StopWordsRemover(stopWords=stopwords, inputCol="words",outputCol="removed")
            wordsData_SW = remover.transform(wordsData)
            hashingTF = HashingTF(inputCol="removed", outputCol="features", numFeatures=20000)
            hashingTF_model = hashingTF.transform(wordsData_SW)
            
            hashingTF = HashingTF(inputCol="words", outputCol="features", numFeatures=20000)
            hashingTF_model = hashingTF.transform(wordsData)
            hashingTF_transfo = MLUtils.convertVectorColumnsFromML(hashingTF_model, "features")
            
            #Vecteur_Apredire=hashingTF_transfo.select("features").take(1)
            #Vecteur_Apredire.show(n=1)
            #P=Vecteur_Apredire[0].features
            
            #P=SparseVector(20000, {4543: 1.0, 5284: 1.0, 8353: 1.0, 8809: 1.0, 10935: 1.0, 10983: 1.0})

            prediction_GBT = GBTModel.predict(hashingTF_transfo.rdd.map(lambda x: x.features))
            prediction=prediction_GBT.take(1)

            comment['Prediction']=prediction[0]
        
            print("Insertion")
            
            # insertion dans MongoDB si Français ou Anglais avec une probabilité à plus de 80%
            db.insert_many(comment[(comment['lg_proba']>0.8) & (comment['langue']=='fr')].to_dict('records'))
    
            return 0
    # Fermeture du client MongoDB
    client.close()

    return 0
# Traitement de tous les RDD reçus dans le streaming en appliquant la fonction précédente
flux.foreachRDD(TraitementCommentaire)

# démarrage du flux
ssc.start()

1
test OK
Insertion
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
test OK
Insertion
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
test OK
Insertion
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
test OK
Insertion
1
test OK
Insertion
0
0
0
0
1
test OK
Insertion
0
0
0
1
test OK
Insertion
0
0
0
1
test OK
Insertion
0
0
1
test OK
Insertion
1
test OK
Insertion
0
0
1
test OK
Insertion
1
test OK
Insertion
1
test OK
Insertion
0
1
test OK
Insertion
1
test OK
Insertion
1
test OK
Insertion
0
0
0
0
0
1
test OK
Insertion
1
test OK
Insertion
1
test OK
Insertion
0
0
0
1
test OK
Insertion
0
0
0
0
1
test OK
Insertion
1
test OK
Insertion
0
0
0
0
0
0
0
0
0
0
0
0
0


## Arrêt du streaming avant la fin du traitement du flux

In [3]:
ssc.stop()
sc.stop()

## Test des données présentes en base


In [13]:
# connection à la base MongoDB
client = MongoClient('localhost:27017')
db=client.test.Comments 


# récupération des données de la collection
data = db.find({"Prediction":0.0})
nb = data.count()
# display the data
print(nb)
for i in data:
    p.pprint(i)

11
{'Prediction': 0.0,
 '_id': ObjectId('5b4caf56b726e81332ab61bc'),
 'comments': 'Un petit nid fouiller douillet situé dans  appartement '
             'tranversant, très lumineux avec une fenêtre dans chaque piece! '
             'Situé au 4ème étage, sans ascenseur, même après une bonne '
             'journée de marche ne nous a pas paru pénalisant. Mais au '
             "contraire, dès l'entrée dans l'appartement,  une belle "
             'décoration, très confortable, bien équipé, un VRAI lit en 160cm '
             'digne de se nom vous, bous fera oublié les marchés montées....un '
             'vrai airbnb, où Anne nous confie son lieux de vie en toute '
             'confiance. \n'
             'Notre hôtes a été très réactive et nous a facilité notre séjour '
             'pour tous les points demandés.\n'
             "Nous reviendrons chez Anne c'est sur si l'occasion se présente à "
             'nous.\n'
             ' Merci encore.',
 'date': 1509667200000,
 'id': 2087

Adapté des scripts du projet GitHub suivant : Stream-Processing-using-PySpark-and-MongoDB--master

https://github.com/vipulkrishnanmd/Stream-Processing-using-PySpark-and-MongoDB-